In [1]:
import pandas as pd

In [17]:
df=pd.read_csv("Luxury_Housing_Bangalore.csv")
df.isnull().sum()

Property_ID                 0
Micro_Market                0
Project_Name                0
Developer_Name              0
Unit_Size_Sqft          10046
Configuration               0
Ticket_Price_Cr         10019
Transaction_Type            0
Buyer_Type                  0
Purchase_Quarter            0
Connectivity_Score          0
Amenity_Score           10090
Possession_Status           0
Sales_Channel               0
NRI_Buyer                   0
Locality_Infra_Score        0
Avg_Traffic_Time_Min        0
Buyer_Comments          18287
dtype: int64

In [ ]:
df['Ticket_Price_Cr'].unique()
df.isnull().sum()
df['Ticket_Price_Cr'].isna().sum()

np.int64(10019)

In [19]:
df['Ticket_Price_Cr'].unique()

array(['12.750846039118798', '16.292151871065954', '10.517724412961911',
       ..., '13.730178633219271', '10.363988981166342',
       '10.415616832095246'], shape=(72855,), dtype=object)

In [29]:
import pandas as pd, re, numpy as np

df = pd.read_csv("Luxury_Housing_Bangalore.csv")

def parse_ticket_price_to_crore(value):
    if pd.isna(value): return np.nan
    s = str(value).strip().lower().replace('₹','').replace('inr','').replace(',','').replace('rs.','').replace('rs','')
    if 'lakh' in s or 'lac' in s:
        nums = re.findall(r'[\d\.]+', s)
        return float(nums[0]) / 100.0 if nums else np.nan
    if 'crore' in s or re.search(r'\bcr\b', s):
        nums = re.findall(r'[\d\.]+', s)
        return float(nums[0]) if nums else np.nan
    nums = re.findall(r'[\d\.]+', s)
    if nums:
        val = float(nums[0])
        if val >= 1e7:        # likely rupees > = 1 crore
            return val / 1e7
        if val >= 1e5:        # likely rupees -> convert
            return val / 1e7
        return val             # assume already in crores (small number)
    return np.nan

df['ticket_price_cr_cleaned'] = df['Ticket_Price_Cr'].apply(parse_ticket_price_to_crore)

# Fill using group median (recommended)
overall_median = df['ticket_price_cr_cleaned'].median()
df['ticket_price_cr_cleaned'] = df.groupby(['Configuration','Micro_Market'])['ticket_price_cr_cleaned'].transform(
    lambda x: x.fillna(x.median())
)
df['ticket_price_cr_cleaned'].fillna(overall_median, inplace=True)

# Replace original column
df.drop(columns=['Ticket_Price_Cr'], inplace=True)
df.rename(columns={'ticket_price_cr_cleaned':'Ticket_Price_Cr'}, inplace=True)

df.to_csv("cleaned_Luxury_Housing_Bangalore_ticket_cleaned.csv", index=False)
print("Done. File saved.")


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_428\2240589987.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ticket_price_cr_cleaned'].fillna(overall_median, inplace=True)


Done. File saved.


In [33]:
df=pd.read_csv("cleaned_Luxury_Housing_Bangalore_ticket_cleaned.csv")
df["Buyer_Comments"].fillna("No comments provided", inplace=True)
df["Buyer_Comments"].isnull().sum()
df.head(10)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_428\2407058162.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Buyer_Comments"].fillna("No comments provided", inplace=True)


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Ticket_Price_Cr
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4bhk,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,yes,9.212491,18,Loved the amenities!,12.750846
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3Bhk,Primary,Other,2024-06-30,4.839024,NaN,Under construction,NRI Desk,no,7.723898,106,No comments provided,16.292152
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4bhk,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,yes,6.985493,113,Agent was not responsive.,10.517724
3,PROP000004,bellary road,Project_3,Embassy,6192.0,3BHK,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,yes,6.100929,106,Excellent location!,9.396367
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4Bhk,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,no,5.312510,18,Too far from my office.,15.345392
5,PROP000006,YELAHANKA,Project_5,Brigade,4290.0,4bhk,Secondary,HNI,2023-09-30,7.936068,NaN,Launch,Online,no,5.154027,16,Underpriced for location.,9.820000
6,PROP000007,Bannerghatta Road,Project_6,Prestige,6776.0,5BHK+,Secondary,Other,2025-03-31,8.643117,8.355238,Under construction,Direct,no,9.791550,74,Too far from my office.,10.896752
7,PROP000008,whitefield,Project_7,Total Environment,5763.0,3BHK,Secondary,NRI,2025-03-31,6.781306,9.565013,Under construction,Direct,yes,9.054713,31,No comments provided,9.357539
8,PROP000009,sarjapur road,Project_8,Prestige,5815.0,3Bhk,Secondary,Other,2024-03-31,7.284164,5.580442,Under construction,NRI Desk,yes,8.552046,68,No comments provided,12.299449
9,PROP000010,JP NAGAR,Project_9,Puravankara,8675.0,4Bhk,Primary,CXO,2023-09-30,9.964399,7.759907,Launch,Direct,yes,8.395691,18,Too far from my office.,9.193310


In [38]:
# Step 1: Check missing count
print("Missing before filling:", df['Unit_Size_Sqft'].isna().sum())

# Step 2: Fill missing by group median
df['Unit_Size_Sqft'] = df.groupby(
    ['Configuration', 'Micro_Market']
)['Unit_Size_Sqft'].transform(lambda x: x.fillna(x.median()))

# Step 3: Fill any leftover NaN (if group median not available)
overall_median = df['Unit_Size_Sqft'].median()
df['Unit_Size_Sqft'].fillna(overall_median, inplace=True)

# Step 4: Verify
print("Missing after filling:", df['Unit_Size_Sqft'].isna().sum())

# Step 5 (Optional): Check average size by configuration
print(df.groupby('Configuration')['Unit_Size_Sqft'].median())


Missing before filling: 10046
Missing after filling: 0
Configuration
3BHK     5989.0
3Bhk     5973.0
3bhk     5998.0
4BHK     5994.5
4Bhk     6008.0
4bhk     5979.0
5BHK+    5986.5
5Bhk+    6028.0
5bhk+    5984.0
Name: Unit_Size_Sqft, dtype: float64


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_428\2558951791.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Unit_Size_Sqft'].fillna(overall_median, inplace=True)
